Peter Dresslar, CAS 520
Module 6 Assignment

### Step 1

The modification adds a small feature allowing the user to control additional (approximately) random link-generation with a toggle and sliders. The new controls are located under the grid display. New code is located in a block demarcated with my name. Further information about the modification appears below. There is a more detailed description of the behavior at the bottom of this page. I have intentionally placed it there in case the reviewer might prefer to evaluate the model before continuing.

### Step 2

I was unable to get the `nw` extension to work at all with my setup. Instead, I added a very basic display for `links-per-node`. A also borrowed this code:

```netlogo
if not plot? [ stop ]
let max-degree max [count link-neighbors] of turtles
plot-pen-reset  ;; erase what we plotted before
set-plot-x-range 1 (max-degree + 1)  ;; + 1 to make room for the width of the last bar
histogram [count link-neighbors] of turtles
```

...from the library model, Preferential Attachment, to generate a histogram plot of the network Degree Distribution. These plots are particularly sensitive to activity from the customization mentioned in Step 1, since additional links can be added over time.

As can be imagined, an increase in the frequency of new link generation not only causes the `links-per-node` summary statistic to grow larger, it also causes our general degree of connectedness to be higher. While I do not think the current model evaluates outcomes based upon link degree classes, it might be interesting to consider how virology might treat such extended relationships, perhaps by thinking about whether specific mutation chances would rise based upon those network measures.

### Step 3

From the supplied images, it appears CV stabilizes at around 100-200 runs for both the susceptable and resistant currencies. I observe that the included "BehaviorSpace" setup has this number at 100, and so we will use that run-number for our experiment (called "Planes" in the .nlogo file.) Weʻll expect to use those same currencies, though we might also peak in on "dead" (survival) as well.

The given variables for BehaviorSpace from the March10 experiment were:

```
["test-frequency" 2 7 14]
["initial-outbreak-size" 10]
["transmitter" true false]
["average-degree" 3 6]
["quarantine-time" 0 7 14]
["morbidity" 10]
["max-recovery-time" 14]
["transmitter-spread" 90]
["transmitter-frequency" 10]
["population" 300]
["virus-spread-chance" 40]
```

It is possible that some of these variables have changed since that version. I will use the following substitutes:

```
["test-frequency" 2 7 14]
["initial-outbreak-size" 10]
["average-degree" 3]
["quarantine-time" 0 7 14]
["morbidity" 10]
["max-recovery-time" 21]
["spreader-reproduction" 90]
["spreader-frequency" 10 25 40]
["non-spreader-reproduction" 10]
["population" 300]
["reproduction" 20]
```

I will add to these variables the following modification-specific variables:

```
["planes-on?" true]
["plane-frequency" 5 15 25]
["plane-radius" 1 2]
```

With the runs complete, all data are located in `M6Model_Dresslar-Planes-table.csv`.

### Step 3 Analysis

We can now load and review the data we have using, to start, pandas for dataframe grouping and analysis.

In [35]:
import pandas as pd

csv_file = "M6Model_Dresslar-Planes-table.csv"

# read datafram in but skip 6 rows of headers
df = pd.read_csv(csv_file, skiprows=6)

# Parameters we want to analyze
params_of_interest = ["plane-frequency", "plane-radius", "spreader-frequency", "quarantine-time", "test-frequency"]

currency_cols = ['susceptable', 'resistant', 'dead', 'quarantine'] # infected is not include since it is automatically 0

# print(df.head())  # note that this is now ALL STEPS. we donʻt want that.

# letʻs create a new df_finals with only the row with the largest value for `step` for each run
# a run is really defined by the combination of all the param_of_interest and the run-number
# just for my benefit weʻll create an "index" column that is a composite of the params_of_interest and the run-number joined by "_"
comp_cols = params_of_interest + ['run-number']
df_finals = df.loc[df.groupby(comp_cols)['step'].idxmax()]
df_finals['index'] = df_finals[comp_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
#print(df_finals.shape)  # should be 10800,something (22ish)
# print(df_finals.head())

# now for each param in params_of_interest, we want to show descriptive stats grouped by that param
for param in params_of_interest:
    print(f"\n\n=== Statistics grouped by {param} ===")
    grouped = df_finals.groupby(param)
    # now each currency, and we just use pandas describe()
    for currency in currency_cols:
        print(f"\n-- {currency} --")
        stats = grouped[currency].describe()
        print(stats)








=== Statistics grouped by plane-frequency ===

-- susceptable --
                  count        mean         std  min   25%    50%    75%  \
plane-frequency                                                            
5                3600.0  144.973889  102.521451  2.0  46.0  118.0  256.0   
15               3600.0  123.690556  112.071727  0.0  24.0   59.0  255.0   
25               3600.0  108.408889  116.966865  0.0  14.0   33.0  257.0   

                   max  
plane-frequency         
5                289.0  
15               288.0  
25               289.0  

-- resistant --
                  count        mean         std  min   25%    50%    75%  \
plane-frequency                                                            
5                3600.0  139.526944   92.414895  9.0  39.0  162.0  228.0   
15               3600.0  158.673889  100.949734  9.0  40.0  216.0  248.0   
25               3600.0  172.370556  105.364430  9.0  39.0  239.0  257.0   

                   max  
plan

To begin with our discussion of the results, it seems encouraging that `plane-frequency` and `plane-radius` have similar directional effects on the endpoint outcomes:

- Greater values for `plane-frequency` and `plane-radius`: Decreased mean `susceptible`
- Greater values for `plane-frequency` and `plane-radius`: Increased mean `resistant`
- Greater values for `plane-frequency` and `plane-radius`: Decreased mean `dead`

We would expect these similar outcomes as each variable leads to increasing network connectivity over time. Moving forward, we might investigate how `plane-frequency` (ignoring `plane-radius` for the moment) interacts with settings for `quarantine-time` and `test-frequency` in particular, since these might be seen as having real world analogues in conjunction with plane travel.

We will borrow some ideas from the following notebook for interaction testing:

https://cambiotraining.github.io/corestats/materials/cs4_practical_two-way-anova.html

In [36]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# still working with df_finals, and letʻs define a pair and cut down our currencies to two.
currency_1 = "susceptable" # lol oh dear i have misspelled susceptible everywhere, forgive me
currency_2 = "resistant"
factors = ['plane-frequency', 'quarantine-time']

# one way anovas (one for each factor)
for factor in factors:  # will print two anovas.
    print(f"\n--- One-Way ANOVA: {currency_1} vs {factor} ---")
    formula_one_way = f"{currency_1} ~ C(Q('{factor}'))"   # ugh, https://stackoverflow.com/questions/50623216/patsy-formula-when-variable-has-a-hypthen
    model_one_way = smf.ols(formula=formula_one_way, data=df_finals).fit()  # kwargs for smf.ols()
    table_one_way = sm.stats.anova_lm(model_one_way, typ = 2)
    print(table_one_way)

# two way anova
print(f"\n--- Two-Way ANOVA (Interaction): {currency_1} vs {' * '.join(factors)} ---")

formula_two_way = f"{currency_1} ~ C(Q('{factors[0]}')) * C(Q('{factors[1]}'))"  # see webpage section 11.5
model_two_way = smf.ols(formula=formula_two_way, data=df_finals).fit()
table_two_way = sm.stats.anova_lm(model_two_way, typ = 2)
print(table_two_way)



--- One-Way ANOVA: susceptable vs plane-frequency ---
                               sum_sq       df          F        PR(>F)
C(Q('plane-frequency'))  2.428211e+06      2.0  99.105331  2.235646e-43
Residual                 1.322703e+08  10797.0        NaN           NaN

--- One-Way ANOVA: susceptable vs quarantine-time ---
                               sum_sq       df            F  PR(>F)
C(Q('quarantine-time'))  5.541765e+07      2.0  3773.572824     0.0
Residual                 7.928089e+07  10797.0          NaN     NaN

--- Two-Way ANOVA (Interaction): susceptable vs plane-frequency * quarantine-time ---
                                                       sum_sq       df  \
C(Q('plane-frequency'))                          2.428211e+06      2.0   
C(Q('quarantine-time'))                          5.541765e+07      2.0   
C(Q('plane-frequency')):C(Q('quarantine-time'))  8.129804e+04      4.0   
Residual                                         7.677138e+07  10791.0   

            

If we can ignore for the moment that I have misspelled "susceptible" many times today, we can observe that these analyses seem to confirm what we can see in the modelʻs operation and what we might expect; that the new `plane-frequency` variable interacts strongly with the `quarantine-time` variable . Our p-value (PR(>F)) appears to be sufficiently low to confirm this. We can try with our `resistant` currency next:

In [37]:
# one way anovas (one for each factor)
for factor in factors:  # will print two anovas.
    print(f"\n--- One-Way ANOVA: {currency_2} vs {factor} ---")
    formula_one_way = f"{currency_2} ~ C(Q('{factor}'))"  
    model_one_way = smf.ols(formula=formula_one_way, data=df_finals).fit()  
    table_one_way = sm.stats.anova_lm(model_one_way, typ = 2)
    print(table_one_way)

# two way anova
print(f"\n--- Two-Way ANOVA (Interaction): {currency_2} vs {' * '.join(factors)} ---")

formula_two_way = f"{currency_2} ~ C(Q('{factors[0]}')) * C(Q('{factors[1]}'))"
model_two_way = smf.ols(formula=formula_two_way, data=df_finals).fit()
table_two_way = sm.stats.anova_lm(model_two_way, typ = 2)
print(table_two_way)


--- One-Way ANOVA: resistant vs plane-frequency ---
                               sum_sq       df          F        PR(>F)
C(Q('plane-frequency'))  1.959488e+06      2.0  98.522779  3.961454e-43
Residual                 1.073691e+08  10797.0        NaN           NaN

--- One-Way ANOVA: resistant vs quarantine-time ---
                               sum_sq       df            F  PR(>F)
C(Q('quarantine-time'))  4.489563e+07      2.0  3761.572115     0.0
Residual                 6.443291e+07  10797.0          NaN     NaN

--- Two-Way ANOVA (Interaction): resistant vs plane-frequency * quarantine-time ---
                                                       sum_sq       df  \
C(Q('plane-frequency'))                          1.959488e+06      2.0   
C(Q('quarantine-time'))                          4.489563e+07      2.0   
C(Q('plane-frequency')):C(Q('quarantine-time'))  6.545682e+04      4.0   
Residual                                         6.240797e+07  10791.0   

                  

And, again, we see the same powerful effects of the two variables, and their combinational effect as well.

It strikes me as an interesting outcome that combining the "link-making" activity of greater `plane-frequency` with the outcomes from `quarantine-time` are so connected; without over-generalizing a toy model adjustment such as "planes", we could imagine the real-world implications and utility of future research into the subject.

### Description of modification

The new planes modification simulates the expansion of contact networks and the resultant spread of virus through airplane travel. It is a toy modification and its sophistication is utterly minimal.

A second breed of turtles has been added, reclassing the existing turtles as `nodes` and the new breed as `planes`. If the `planes-on?` toggle is set to On, planes will randomly emanate from one of the four "walls" of the arena and head on a random vector to some other wall. Along the way, any nodes within `plane-radius` of the plane are connected to *every other node* the plane has already come into contact with, following the linking rules of the base model (for instance, "dead" nodes are not linked). Higher values of `plane-radius` can thus generate very large numbers of network connections.

The model otherwise functions as normal, and disease can be seen to travel across these new, airplane-generated links, just as would be expected with the real world phenomenon.